In [1]:
import sys 
import torch
import random
import numpy as np
import pandas as pd
import gc
import time
import random
from tqdm import tqdm

from IPython.display import display

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoModel

if (not torch.cuda.is_available()): print("Sorry - GPU required!")
    
import logging
logging.getLogger('transformers').setLevel(logging.ERROR)

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

In [2]:
test_df = pd.read_csv('/kaggle/input/llms-you-cant-please-them-all/test.csv')
test_df

,id,topic
0,1097671,Compare and contrast the importance of self-reliance and adaptability in healthcare.
1,1726150,Evaluate the effectiveness of management consulting in addressing conflicts within marketing.
2,3211968,Discuss the role of self-reliance in achieving success in software engineering.


In [3]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()
for obj in ['model', 'pipe', 'tokenizer']:
    if obj in globals():
        del globals()[obj]

model_name = '/kaggle/input/phi-3.5-mini-instruct/pytorch/default/1'
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
max_new_tokens = 130  # Maximum length of generated text
word_count_to_request = 60   #We ask the model for this many words as part of the prompt prefix

temperature = 0.9     # Higher temperature = more random/creative outputs
top_p = 0.9          # Nucleus sampling parameter for more diverse outputs

In [5]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    trust_remote_code=True,
    max_new_tokens=max_new_tokens,
    temperature=temperature,
    top_p=top_p,
    do_sample=True)

In [6]:
def get_response(messages):
    response = pipe(messages)[0]['generated_text'][-1]['content']
    
    response = response.strip()
    
    words = response.split()
    
    if len(words) > 100:
        response = " ".join(words[:100])
    
    last_period = response.rfind('.')
    last_question = response.rfind('?') 
    last_exclamation = response.rfind('!')
    last_close_bracket = response.rfind(']')

    last_punct = max(last_period, last_question, last_exclamation, last_close_bracket)
    
    if last_punct != -1:
        return response[:last_punct + 1]
    return response

In [19]:
messages_list = [
    {"role": "user", "content": f"Write an essay about {row['topic']}. Write your answer in 50 words, and then write a new one (same size) about what happened on a day of your choice, but be coherent, and do not repeat the same day in every case."} 
    for _, row in test_df.iterrows()]

responses = [get_response([msg]) for msg in messages_list]

test_df["essay"] = responses

In [20]:
submission = test_df.copy()
submission = submission[['id', 'essay']]
submission

,id,essay
0,1097671,"Self-reliance empowers healthcare professionals to trust their expertise, ensuring quality care. Adaptability, however, is crucial for responding to unforeseen challenges, enhancing patient outcomes. Both traits are indispensable, with self-reliance fostering confidence and adaptability ensuring flexibility in dynamic healthcare environments.\n\nOn a sunny Thursday afternoon, the local library hosted a community book club meeting. Members gathered around, discussing their latest read, ""The Great Gatsby."
1,1726150,"Management consulting enhances conflict resolution in marketing by providing expert strategies, fostering open communication, and facilitating stakeholder alignment. It offers objective insights, identifies underlying issues, and proposes tailored solutions, thus optimizing marketing effectiveness.\n\nOn April 10th, a local community garden initiative faced a dispute between volunteers over resource allocation. A management consultant organized a workshop, encouraged dialogue, and developed a transparent volunteer system. The conflict dissolved, cooperation flourished, and the garden's productivity soared."
2,3211968,"Self-reliance in software engineering fosters innovation, continuous learning, and problem-solving skills, essential for navigating complex tech landscapes and achieving professional success. Embracing challenges independently cultivates resilience and adaptability.\n\nOn April 10th, amidst the spring blossoms, I visited the local botanical garden, marveling at the interconnectedness of nature's diversity. Each flower and tree, unique yet part of a larger ecosystem, inspired reflection on the interconnectedness of our daily experiences and learning opportunities."


In [21]:
submission.to_csv('submission.csv', index=False)